In [373]:
#Taking csv files(Execution traces) as input
import csv
import re
csv_arr = []
n = input('Enter no of csv files: ')
for i in range(int(n)):
    csv_arr.append(input('Enter the paths: '))
dataset1 = []
c = []
for i in csv_arr:
    path = i
    file = open(path, newline = '')
    reader = csv.reader(file)
    data = [row for row in reader]
    c.append(len(data))
    dataset1.append(data)

Enter no of csv files: 4
Enter the paths: case2_1.csv
Enter the paths: case2_2.csv
Enter the paths: case2_3.csv
Enter the paths: case2_4.csv


In [374]:
# Counting the number of lines in the execution traces of both server and client
client_count = []
client_sync = []
server_count = len(dataset1[0])
for i in range(1, int(n)):
    client_count.append(len(dataset1[i]))
    client_sync.append(0)
print(server_count)
print(client_count)

250
[59, 57, 57]


In [375]:
#Identifying the connected clients through the port no
no_of_clients = int(n) - 1
port_list = [r'port\=\d{5}', r'localport=\d{5}']
server_port = []
client_port = []
socket_port = []
server = []
connected_clients = []
flg_count = 0
c = 0
p = re.compile(port_list[0])
q = re.compile(port_list[1])
for i in range(server_count):
    for j in dataset1[0][i]:
        if p.findall(j):
            server_port.append(dataset1[0][c][6])
            if flg_count == 0:
                server.append(dataset1[0][c][4])
                flg_count = 1
#             server_port.append(c+1)
            socket_port.append(c+1)
    c = c + 1

    
for i in range(no_of_clients):
    c1 = 0
    for j in range(client_count[i]):
        for k in dataset1[i+1][j]:
            if q.findall(k):
                connected_clients.append(dataset1[i+1][c1][4])
                client_port.append(dataset1[i+1][c1][7])
#                 client_port.append(dataset[i+1][c1][4])
#                 client_port.append(c1+1)
        c1 = c1 + 1
    
    
print(server_port)
server_port_distinct = []
[server_port_distinct.append(x) for x in server_port if x not in server_port_distinct]
print(server_port_distinct)
print(client_port)
# print(server_port[0][5:10])
# print(client_port[0][10:15])
# del client_port[1], client_port[2], client_port[3]
print(server)
print(socket_port)
print(connected_clients)

['port=46828', 'port=46832', 'port=46836', 'port=46836', 'port=46832', 'port=46828']
['port=46828', 'port=46832', 'port=46836']
['localport=46828]"""', 'localport=46832]"""', 'localport=46836]"""']
[' " context=ServerA#main:1']
[17, 81, 143, 209, 225, 241]
[' " context=Client.ClientA#main:1', ' " context=ClientA1#main:1', ' " context=ClientA2#main:1']


In [376]:
server_text = []
flag = 0
for i in range(len(server[0])):
    if server[0][i] == '#':
        tem = i
    if server[0][i] == '.':
        tem1 = i
        flag = 1
if flag == 1:
    server_text.append(server[0][tem1+1:tem])
else:
    server_text.append(server[0][11:tem])
print(server_text)

['ServerA']


In [377]:
unique = []
client_start = []
client_end = []
c = 0
for x in server_port: 
    if x not in unique:
        unique.append(x)
        client_start.append(c)
        for i in range(c+1, len(server_port)):
            if(x == server_port[i]):
                client_end.append(i)
    c = c + 1
print(client_start)
print(client_end)

[0, 1, 2]
[5, 4, 3]


In [378]:
client_startpoints = []
for i in range(len(client_start)):
    client_startpoints.append(socket_port[client_start[i]])
print(client_startpoints)
client_endpoints = []
for i in range(len(client_end)):
    client_endpoints.append(socket_port[client_end[i]])
print(client_endpoints)    

[17, 81, 143]
[241, 225, 209]


In [379]:
#Checks whether a new client communicates to the server before an existing client finishes its communication 
#If no new client comes before an existing client then sync is set to true else sync is set to false 
sync = True
for i in range(0, len(server_port) - 1, 2):
    if(server_port[i] != server_port[i+1]):
        sync = False
        break
print(sync)

False


In [380]:
merge_list = []
for i in range(len(server_port_distinct)):
    for j in range(len(client_port)):
        if server_port_distinct[i][5:10] == client_port[j][10:15]:
            merge_list.append(j + 1)
            break
print(merge_list)   

[1, 2, 3]


In [381]:
#Identifies the order in which the clients communicate to the server and stores them in a list
connected_clients_sync = []
for i in range(len(merge_list)):
    connected_clients_sync.append(connected_clients[merge_list[i]- 1])
print(connected_clients_sync)

[' " context=Client.ClientA#main:1', ' " context=ClientA1#main:1', ' " context=ClientA2#main:1']


In [382]:
client_text = []
print(connected_clients[0][1:10])
for i in range(len(connected_clients_sync)):
    flag = 0
    for j in range(len(connected_clients_sync[i])):
        if connected_clients_sync[i][j] == '#':
            tem = j
        if connected_clients_sync[i][j] == '.':
            tem1 = j
            flag = 1
    if flag == 1:
        client_text.append(connected_clients_sync[i][tem1+1:tem])
    else:
        client_text.append(connected_clients_sync[i][11:tem])
print(client_text)

" context
['ClientA', 'ClientA1', 'ClientA2']


In [383]:
Client_Dict = {}
for i in range(len(client_text)):
    Client_Dict[client_text[i]] = 0
print(Client_Dict)

{'ClientA': 0, 'ClientA1': 0, 'ClientA2': 0}


In [384]:
#The csv files are arranged in the hierarchical order of clients
csv_arr_ordered = []
csv_arr_ordered.append(csv_arr[0])
for i in merge_list:
    for j in range(len(csv_arr)):
        if i == j:
            csv_arr_ordered.append(csv_arr[j])
            break
print(csv_arr_ordered)

['case2_1.csv', 'case2_2.csv', 'case2_3.csv', 'case2_4.csv']


In [385]:
dataset = []
client_count = []
for i in csv_arr_ordered:
    path = i
    file = open(path, newline = '')
    reader = csv.reader(file)
    data = [row for row in reader]
    client_count.append(len(data))
    dataset.append(data)

In [386]:
print(dataset[1][0])
print(client_count)
del client_count[0]


['SYSTEM', ' "1"', ' "SYSTEM"', ' "System Start"', ' " "']
[250, 59, 57, 57]


In [387]:
#Identifies the points at which the server communicates to the client and stores them in a list called count_list
re_list = [r'port\=\d{5}',
           '\s[a-zA-Z0-9]+\=java.net.SocketInputStream',
           '\s[a-zA-Z0-9]+\=java.net.SocketOutputStream',
#            '\s[a-zA-Z0-9]+\=java.util.Scanner',
           '\s[a-zA-Z0-9]+\=java.io.PrintStream',
           '\s[a-zA-Z0-9]+\=java.io.DataInputStream',
           '\s[a-zA-Z0-9]+\=java.io.DataOutputStream',
#            '\s[a-zA-Z0-9]+\=java.io.BufferedReader'
          ]

net_socket = [r'\s[a-zA-Z0-9]+\=java.net.Socket']
q = re.compile(net_socket[0])



count_list = []
flg = 0
for k in range(len(re_list)):
    count = 1
    p = re.compile(re_list[k])
    for i in range(server_count):
        count1 = 0
        for j in dataset[0][i]:
               if p.findall(j):
                    count1 = count
                    print(j)
        count=count+1
        if count1 == 0:
            continue
        else:
            count_list.append(count1)


        
count_list.sort()
count_list.append(server_count - 1)
# count_list.insert(7, 110)
# count_list.insert(15, 203)
# del count_list[len(count_list) - 1]
print(count_list)


port=46828
port=46832
port=46836
port=46836
port=46832
port=46828
 dis=java.io.DataInputStream (id=62)"
 dis=java.io.DataInputStream (id=74)"
 dis=java.io.DataInputStream (id=81)"
 dis=java.io.DataInputStream (id=87)"
 dis=java.io.DataInputStream (id=94)"
 dis=java.io.DataInputStream (id=100)"
 dos=java.io.DataOutputStream (id=59)"
 dos=java.io.DataOutputStream (id=73)"
 dos=java.io.DataOutputStream (id=80)"
 dos=java.io.DataOutputStream (id=86)"
 dos=java.io.DataOutputStream (id=93)"
 dos=java.io.DataOutputStream (id=99)"
[17, 51, 58, 72, 77, 81, 112, 120, 134, 139, 143, 173, 182, 196, 201, 209, 225, 241, 249]


In [388]:
def merge(client_startpoints, client_endpoints): 
    merged_list = [(client_startpoints[i], client_endpoints[i]) for i in range(0, len(client_startpoints))] 
    return merged_list
client_breakpoints = []
client_breakpoints = merge(client_startpoints, client_endpoints)
print(client_breakpoints)

[(17, 241), (81, 225), (143, 209)]


In [389]:
print(client_sync)
client_sync_count = 0
for i in range(len(client_breakpoints) - 1):
    if(client_breakpoints[i][1] > client_breakpoints[i+1][0]):
        client_sync[i] = 1
        client_sync_count = client_sync_count + 1
print(client_sync)
print(client_sync_count)
print(client_breakpoints)
print(Client_Dict)

[0, 0, 0]
[1, 1, 0]
2
[(17, 241), (81, 225), (143, 209)]
{'ClientA': 0, 'ClientA1': 0, 'ClientA2': 0}


In [390]:
server_socket_breakpoints = []
x = 0
y = 0
for i in range(len(client_breakpoints)):
    for j in client_breakpoints[i]:
        for k in range(len(count_list)):
            if j == count_list[k]:
                server_socket_breakpoints.append(k)
print(server_socket_breakpoints)
print(server_socket_breakpoints)
print(server_socket_breakpoints)

[0, 17, 5, 16, 10, 15]
[0, 17, 5, 16, 10, 15]
[0, 17, 5, 16, 10, 15]


In [391]:
server_startpoints = []
server_endpoints = []
for i in range(len(server_socket_breakpoints)):
    if i % 2 == 1:
        server_endpoints.append(server_socket_breakpoints[i])
    else:
        server_startpoints.append(server_socket_breakpoints[i])
server_endpoints_index = []
server_endpoints_index.extend(server_endpoints)
server_endpoints.sort()
print(server_endpoints)
for i in range(1, len(server_endpoints) - 1):
    if server_endpoints[0] != server_endpoints[1] - 1:
        del server_endpoints[0]
        break
    if server_endpoints[i] != (server_endpoints[i+1] - 1):
        del server_endpoints[i+1]
    print(i)
    
print(server_endpoints)
print(server_endpoints_index)
print(server_startpoints)

[15, 16, 17]
1
[15, 16, 17]
[17, 16, 15]
[0, 5, 10]


In [392]:
server_end_check = []
for i in range(len(server_endpoints_index)):
    server_end_check.append(0)
for i in range(len(server_endpoints_index) - 1):
    if server_endpoints_index[i] < server_endpoints_index[i+1] and abs(server_endpoints_index[i]-server_endpoints_index[i+1]) == 1:
        server_end_check[i + 1] = 1
print(server_end_check)

[0, 0, 0]


In [393]:
def bubbleSort(arr, pos):
    n = len(arr)
    # Traverse through all array elements
    for i in range(pos, n):
        ctr = 0
        # Last i elements are already in place
        for j in range(pos, n-ctr-1):
 
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if arr[j] > arr[j+1] :
                arr[j], arr[j+1] = arr[j+1], arr[j]
            ctr = ctr + 1

In [394]:
sample = [5, 3, 10, 15, 9, 1, 4]
bubbleSort(sample, 2)
print (sample) 


[5, 3, 1, 4, 9, 10, 15]


In [395]:
def getCount(arr, n, num1, num2): 
      
    # Find num1 
    for i in range(0,n): 
        if (arr[i] == num1): 
            break
              
    #If num1 is not present or present at end 
    if (i >= n-1): 
        return 0
  
    # Find num2 
    for j in range(n-1, i, -1): 
        if (arr[j] == num2): 
            break
  
    # If num2 is not present 
    if (j == i): 
        return 0
  
    # return number of elements between 
    # the two elements. 
    return (j-i-1)  

In [396]:
print(client_endpoints)

[241, 225, 209]


In [397]:
for i in range(len(client_endpoints)):
    Client_Dict[client_text[i]] = client_endpoints[i]
print(Client_Dict)

{'ClientA': 241, 'ClientA1': 225, 'ClientA2': 209}


In [398]:
client_end_check = []
for i in range(len(client_endpoints)):
    client_end_check.append(0)
for i in range(len(client_endpoints) - 1):
    if client_endpoints[i] > client_endpoints[i+1]:
        client_end_check[i] = 1
print(client_end_check)

[1, 1, 0]


In [399]:
client_endpoints.sort()
for i in range(len(client_endpoints)):
    for j in range(len(client_endpoints)):
        if Client_Dict[client_text[i]] == client_endpoints[j]:
            Client_Dict[client_text[i]] = j+1
print(client_endpoints)
print(Client_Dict)

[209, 225, 241]
{'ClientA': 3, 'ClientA1': 2, 'ClientA2': 1}


In [400]:
print(client_sync)
print(client_end_check)

[1, 1, 0]
[1, 1, 0]


In [401]:
# Driver Code
count_between_breakpoints = []
count_length = len(count_list)
for i in range(len(client_endpoints)):
    if client_sync[i] == 0 or client_end_check[i] == 0:
        continue
    num1 = client_endpoints[i]
    num2 = client_endpoints[i+1]
    count_between_breakpoints.append(getCount(count_list, count_length, num1, num2))
    if num2 == client_endpoints[len(client_endpoints) - 1]:
        break

print(count_between_breakpoints)

[0, 0]


In [402]:
#Identifies the points at which the clients communicate to the server and stores them in a list called count_list1
re_list = [r'localport=\d{5}',
           '\s[a-zA-Z0-9]+\=java.net.SocketInputStream',
           '\s[a-zA-Z0-9]+\=java.net.SocketOutputStream',
#            '\s[a-zA-Z0-9]+\=java.util.Scanner',
           '\s[a-zA-Z0-9]+\=java.io.PrintStream',
           '\s[a-zA-Z0-9]+\=java.io.DataInputStream',
           '\s[a-zA-Z0-9]+\=java.io.DataOutputStream',
#            '\s[a-zA-Z0-9]+\=java.io.BufferedReader'
          ]



#Splits the execution trace of server according to the points stored in count_list 
def server_split(c1, c2):
    temp = []
    temp1 = []
    for i in range(c1 , c2 + 1):
        for j in dataset[0][i]:
            if i == c1:
                temp.append(j)
            elif j=="main" or j=="SYSTEM" or j=="pool-1-thread-1" or j=="pool-1-thread-2" or j == "Thread-0" or j == "Thread-1" or j == "Thread-2":
                temp1.append(temp)
                temp=[]
                temp.append(j)
            else:
                temp.append(j)
    return temp1


c_list = []
client_length = []
pos = 0

for z in range(1, int(n)):
    pos1 = pos
    for k in range(len(re_list)):
        p = re.compile(re_list[k])
        count = 1
        for i in range(len(dataset[z])):
            count1 = 0
            for j in dataset[z][i]:
                if p.findall(j):
                    count1 = count
#                     print(j)
            count=count+1
            if count1 == 0:
                continue
            else:
                if j == ' client=java.net.Socket (id=29)"':
                    c_list.append(count1)
                    break
                c_list.append(count1)
                pos = pos + 1
    c_list.append(client_count[z - 1] - 1)
    pos = pos + 1
    client_length.append(pos)
    if z == 1:
        c_list.sort()
    else:
        bubbleSort(c_list, pos1 + 1)
        
        

    
print(c_list)
print(client_length)


count_list1 = []


count_list1.extend(c_list)
print(count_list1)

#Splits the execution traces of the clients according to the points stored in count_list1
def client_split(c1, c2, z):
    temp = []
    temp1 = []
    for i in range(c1 , c2 + 1):
        for j in dataset[z][i]:
            if i == c1:
                temp.append(j)
            elif j=="main" or j=="SYSTEM":
                temp1.append(temp)
                temp=[]
                temp.append(j)
            else:
                temp.append(j)
    return temp1 
 
# a = []
# a.append(server_split(0, count_list[0]))
# import numpy as np
# np.shape(server_split(count_list[0],count_list[1]))

[19, 21, 26, 38, 43, 58, 17, 19, 24, 36, 41, 56, 17, 19, 24, 36, 41, 56]
[6, 12, 18]
[19, 21, 26, 38, 43, 58, 17, 19, 24, 36, 41, 56, 17, 19, 24, 36, 41, 56]


In [403]:
temp = []
stream = []
for i in range(len(server_socket_breakpoints) - 1):
    if i % 2 == 1:
        if server_socket_breakpoints[i] > server_socket_breakpoints[i+1]:
            temp.append(server_socket_breakpoints[i+1])
print(temp)
count = 0
print(len(count_list))
temp_list = []
temp_list.extend(server_endpoints_index)
temp_list.sort(reverse=True)
print(temp_list)
for i in range(len(count_list)):
    if len(temp) == 0:
        break
    if i == temp[count]:
        k = 0
        temp_var = i
        for j in temp_list:
            if j == temp_var - 1:
                k = k + 1
                temp_var = temp_var - 1
        print(k)
        stream.append((i - k) - 3)
        count = count + 1
        if(count == len(temp)):
            break
        
print(stream)
list_order = [3, 4, 5, 0, 1, 2]


print(stream)

[5, 10]
19
[17, 16, 15]
0
0
[2, 7]
[2, 7]


In [404]:
for i in range(len(stream) - 1):
    if client_sync[i] != 1 or client_end_check[i] != 0:
        stream[i], stream[i+1] = stream[i+1], stream[i]        
print(stream)

[7, 2]


In [405]:
server_client_finish_point = []
for i in range(len(client_length)):
    server_client_finish_point.append(0)
print(server_client_finish_point)

[0, 0, 0]


In [406]:
#Client length stores the points at which a new client communicates to the server 
print(client_length)
client_length1 = []
append_count = 0

if sync == False:
    if len(count_between_breakpoints) != 0:
        k = 2 + count_between_breakpoints[0]
    else:
        k = 2
    for i in range(len(client_length)):
        if client_sync[i] == 1:
            append_count = append_count + 1
            client_length1.append(client_length[i] - k)
            k = k + 2
        else:
            if len(count_between_breakpoints) == 0:
                client_length1.append(client_length[i] - (append_count*2))
            else:
                client_length1.append(client_length[i] - (append_count*2+count_between_breakpoints[0]))
            if(server_end_check[i] == 1):
                server_client_finish_point[i] = client_length1[i]
                client_length1[i] = client_length1[i] - 2

print(client_length)
print(client_length1)
print(server_client_finish_point)


[6, 12, 18]
[6, 12, 18]
[4, 8, 14]
[0, 0, 0]


In [407]:
server_socket_breakpoints.sort()
print(server_socket_breakpoints)

[0, 5, 10, 15, 16, 17]


In [408]:
thread_list = []
actual_client_endpoints = []
for i in range(no_of_clients):
    thread_list.append('Thread-'+ str(i))
print(thread_list)

for i in range(len(thread_list)):
    if server_end_check[i] == 1:
        thread_list[i-1], thread_list[i] = thread_list[i], thread_list[i-1]
print(thread_list)
client_sync_count_temp = no_of_clients - 1
if sync == False:
    for j in range(no_of_clients-1, 0, -1):
        for i in range(server_count - 1):
            if dataset[0][i][0] == thread_list[j] and dataset[0][i+1][0] == thread_list[j - 1]:
                if client_sync[j - 1] != 1 or client_end_check[j - 1] != 0:
                    actual_client_endpoints.append(i+1)
                break
            
print(actual_client_endpoints)

['Thread-0', 'Thread-1', 'Thread-2']
['Thread-0', 'Thread-1', 'Thread-2']
[218, 234]


In [409]:
k = 0
for j in actual_client_endpoints:
    for i in range(len(count_list)-1):
        if count_list[i] > j:
            count_list.insert(i, j)
            del count_list[i-1]
            break
print(count_list)

[17, 51, 58, 72, 77, 81, 112, 120, 134, 139, 143, 173, 182, 196, 201, 218, 234, 241, 249]


In [410]:
print(client_sync)
print(client_end_check)

[1, 1, 0]
[1, 1, 0]


In [411]:
#Merges the execution traces of server and clients in the case where no new client communicates to the server during an
#existing client server communication
if sync == True:
    merge = []
    merge_flag = 0
    client_counter = 1
    for i in range(len(count_list1)):
        if i == 0:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(0, count_list[0]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[0], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(0, count_list1[0], client_counter))
            merge.append([[client_text[0], count_list1[0] + 4,' ','Method Exit']])
        elif (i+1) % client_length[client_counter - 1] == 0:
            merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
            merge.append(client_split(count_list1[i-1], count_list1[i], client_counter))
            merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[i-1], count_list[i]))
            if client_counter == len(client_length):
                merge.append(server_split(count_list[len(count_list)- 2], count_list[len(count_list) - 1]))
            merge.append([[server_text[0], 1,' ','Method Exit']])
            merge_flag = 1
            client_counter = client_counter + 1
            
        elif i == len(count_list1) - 1:
            merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
            merge.append(client_split(count_list1[i-1], count_list1[i], client_counter))
            merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[i-1], count_list[i]))
            merge.append([[server_text[0], 1,' ','Method Exit']])
            break
        elif merge_flag == 1:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[i-1], count_list[i]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(0 , count_list1[i], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 4,' ','Method Exit']])
            merge_flag = 0
        else:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[i-1], count_list[i]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(count_list1[i-1], count_list1[i], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 4,' ','Method Exit']])
            

In [412]:
#Merges the execution traces of server and clients in the case where a new client communicates to the server during an
#existing client server communication
if sync == False:
    merge = []
    merge_flag = 0 #Used to start from a new client
    client_counter = 1
    client_sync_count1 = client_sync_count
    client_length_count = 0
    server_iter = 0
    client_iter = 0
    i = 0
    if len(count_between_breakpoints) != 0:
        k = 1 + count_between_breakpoints[0]
    else:
        k = 1
    client_end_flag = 0
    normal_flag = True # Merges the client and the server normally at the end
    immediate_flag = True # work in progress
    client_end = False # Used to check whether a client ends before another client
    client_end_enter_count = 0
    stream_iter = 0
    server_index = 0
    server_break_iter = 0
    client_break_iter = 0
    client_counter_temp = 0
    count_between_breakpoints_iter = 0
    while(1):
        if i == 0:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(0, count_list[0]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(0, count_list1[0], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 4,' ','Method Exit']])
            print(f'{i} 0')
            i = i + 1
            server_iter = server_iter + 1
            client_iter = client_iter + 1
        
        elif client_end_flag == 1:
            print('In client end flag')
            if client_end == True:
                for i in range(2):
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                    merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                    merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_endpoints_index[stream_iter]-1], count_list[server_endpoints_index[stream_iter]]))
                merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                client_length_count += 1
                client_sync_count1 -= 1
                if client_length_count == len(client_length1) and client_sync_count1 == 0:
                    merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
                    break
                continue
            server_iter = stream[stream_iter] + 1
            client_iter = stream[stream_iter] + client_sync_count1-1 + 1 + client_end_enter_count #not sure
            stream_iter = stream_iter + 1
            client_counter = client_counter_temp - 1
            client_counter_temp = client_counter_temp - 1
            print(f'{client_counter} CLIENT COUNTER before IN')
            for j in range(0, 2):
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                print("IN")
            print(f'{i}Value of i')
                        
            if immediate_flag == False:
                server_iter = server_break_iter + 1
                for i in range(count_between_breakpoints[0]):
                    print('In immediate false')
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                    merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                    merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], count_list1[0] + 10,' ','Method Exit']])
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                    
                
            client_sync_count1 = client_sync_count1 - 1
            merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
            print(f'{client_iter}--')
            merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
            print("OUT")
            merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
            merge.append([[server_text[0], 1,' ','Method Call']])
            if len(count_between_breakpoints) != 0:
                merge.append(server_split(count_list[server_endpoints[server_index]+count_between_breakpoints[0]], count_list[server_endpoints[server_index+1]]))
            else:
                merge.append(server_split(count_list[server_endpoints[server_index]], count_list[server_endpoints[server_index+1]]))
                
            server_index = server_index + 1
            normal_flag = True
            if client_length_count == len(client_length1) and client_sync_count1 == 0:
                merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
            if client_sync_count1 == 0:
                merge_flag = 1
                client_end_flag = 0
                client_counter = client_length_count + 1
                print(f'{client_counter} client counter')
                print(f'{server_break_iter} server_break_iter')
                print(f'{client_break_iter} client_break_iter')
                server_iter = client_break_iter + 1
                client_iter = client_break_iter + 1
                print(f'{client_iter} client iter')
                print(f'{server_iter} server_iter')
                i = i + 1
                if client_length_count == len(client_length1):
                    break
                continue
                
        elif (i+1) % client_length1[client_counter - 1] == 0 or i == server_client_finish_point[client_counter - 1]:
            print('In elif.....client_length1[client_counter - 1]')
            print(f'{i}....Value of i In elif.....client_length1[client_counter - 1] ')
            if client_sync[client_counter - 1] == 1:
                print('In client_sync[client_counter - 1] == 1')
                server_break_iter = server_iter
                client_break_iter = client_iter
                client_counter_temp = client_counter
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for j in range(2):
                        merge.append([[server_text[0], 1,' ','Method Call']])
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                        merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                        merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                        merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
                    stream_iter += 1
                    merge.append([[server_text[0], 1,' ','Method Exit']])
                    client_counter  = client_counter_temp + 1 # not sure
                    client_counter_temp = client_counter #not sure
                    server_iter = server_startpoints[client_counter - 1]
                    client_iter = client_length[client_counter - 2]
                    client_end = False
                    merge_flag = 1
                    i = i + 1
                    client_length_count += 1
                    client_sync_count1 -= 1
                    if client_length_count == len(client_length) and client_sync_count1 == 0:
                        break
                    continue
                    
                if client_end_check[client_counter - 1] == 0:
                    client_end = True
                normal_flag = False
                server_iter = client_length[client_counter - 1] - k
                k = k + 1
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 1')
                client_iter = client_length[client_counter - 1]
                print(f'{client_iter} client_iter In client_sync[client_counter - 1] == 1')
                client_counter = client_counter + 1
                print(f'{i} value of i +')
                i = i + 1
                merge_flag = 1
                client_length_count += 1
                client_counter_temp = client_counter
                continue
            if normal_flag == True:
                print('In normal flag')
                merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
                if client_counter == len(client_length):
                    merge.append(server_split(count_list[len(count_list)- 2], count_list[len(count_list) - 1]))
                merge.append([[server_text[0], 1,' ','Method Exit']])
                client_length_count += 1
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                i = i + 1 
                if client_length_count == len(client_length1):
                    break
                merge_flag = 1
                client_counter +=1
                continue
                
            if client_sync[client_counter - 1] == 0:
                print('In client_sync[client_counter - 1] == 0')
                server_break_iter = server_iter
                client_break_iter = client_iter
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 0')
                print(f'{client_iter} client_iter in client_sync[client_counter - 1] == 0')
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for i in range(2):
                        merge.append([[server_text[0], 1,' ','Method Call']])
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                        merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                        merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                        merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
                    stream_iter += 1
                    merge.append([[server_text[0], 1,' ','Method Exit']])
                    client_counter +=1
                    server_iter = server_break_iter
                    client_iter = client_break_iter
                    client_end_flag = 1
                    continue
                client_length_count += 1
                merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
                merge.append([[server_text[0], 1,' ','Method Exit']])
                client_end_flag = 1
                client_counter = client_counter - 1
                if len(count_between_breakpoints) > 0 and count_between_breakpoints[count_between_breakpoints_iter] > 0:
                    immediate_flag = False
                    count_between_breakpoints_iter += 1
                
                
        elif merge_flag == 1:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(0 , count_list1[client_iter], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 4,' ','Method Exit']])
            merge_flag = 0
            print(f'{server_iter} server_iter in merge_flag')
            print(f'{client_iter} client_iter in merge_flag')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i} value of i in elif merge flag')
            i = i + 1
        else:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 10,' ','Method Exit']])
            print(f'{server_iter} server_iter else')
            print(f'{client_iter} client_iter else')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i}else i')
            i = i + 1

0 0
1 server_iter else
1 client_iter else
1else i
2 server_iter else
2 client_iter else
2else i
In elif.....client_length1[client_counter - 1]
3....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
5 server_iter in client_sync[client_counter - 1] == 1
6 client_iter In client_sync[client_counter - 1] == 1
3 value of i +
5 server_iter in merge_flag
6 client_iter in merge_flag
4 value of i in elif merge flag
6 server_iter else
7 client_iter else
5else i
7 server_iter else
8 client_iter else
6else i
In elif.....client_length1[client_counter - 1]
7....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
10 server_iter in client_sync[client_counter - 1] == 1
12 client_iter In client_sync[client_counter - 1] == 1
7 value of i +
10 server_iter in merge_flag
12 client_iter in merge_flag
8 value of i in elif merge flag
11 server_iter else
13 client_iter else
9else i
12 server_iter else
14 client_iter e

In [413]:
#function definition
def uml_insert(merge, k, j, uml, start, end):
    global tem
    if merge[k][j][3] == ' "Method Call"':
        s = merge[k][j][4]
        flg = 0
        for c in range(len(s)):
            if s[c] == '#':
                tem = c
            if s[c] == '.':
                tem1 = c
                flg = 1
        if flg == 0:
            s = s[10:tem]
        else:
            s = s[tem1+1:tem]
        s = s.replace(":",".")
        uml.append(s)
        start.append(s)
        uml.append("->")
        s1 = merge[k][j][5]
        flg1 = 0
        for c in range(len(s1)):
            if s1[c] == '#':
                tem = c
            if s1[c] == '.':
                tem1 = c
                flg1 = 1
        if flg1 == 0:
            message = s1[tem+1:-1]
            s1 = s1[8:tem]
        else:
            message = s1[tem+1:-1]
            s1 = s1[tem1+1:tem]
        s1 = s1.replace(":",".")
        uml.append(s1)
        uml.append(":")
        uml.append(message)
        uml.append("newline")
        uml.append(f'activate {s1}')
        uml.append("newline")
    if merge[k][j][3] == ' "Method Exit"':
        s2 = merge[k][j][4]
        flg2 = 0
        for c in range(len(s2)):
            if s2[c] == '#':
                tem = c
            if s2[c] == '.':
                tem1 = c
                flg2 = 1
        if flg2 == 0:
            s2 = s2[12:tem]
        else:
            s2 = s2[tem1+1:tem]
        s2 = s2.replace(":",".")
        end.append(s2)
        uml.append(end[len(end) - 1])
        uml.append("-->")
        uml.append(start[len(start) - 1])
        start.remove(start[len(start) - 1])
        uml.append("newline")
        uml.append(f'deactivate {end[len(end) - 1]}')
        uml.append("newline")
def socket_insert_request(merge, k, socket):
    for o in range(2):
        socket.append(merge[k][0][0])
        if o == 0:
            socket.append("->")
        else:
            socket.append(":sends request")
            socket.append("newline")
        k+=1
def socket_insert_response(merge, k, socket):
    for m in range(2):
        socket.append(merge[k][0][0])
        if m == 0:
            socket.append("-->")
        else:
            socket.append(":sends response")
            socket.append("newline")
        k+=1

In [414]:
#Used for sequence diagram generation
#Merges the execution traces of server and clients in the case where a new client communicates to the server during an
#existing client server communication
if sync == False:
    uml = ["@startuml"]
    start = []
    end = []
    socket = []
    merge = []
    merge_flag = 0 #Used to start from a new client
    client_counter = 1
    client_sync_count1 = client_sync_count
    client_length_count = 0
    server_iter = 0
    client_iter = 0
    i = 0
    if len(count_between_breakpoints) != 0:
        k = 1 + count_between_breakpoints[0]
    else:
        k = 1
    client_end_flag = 0
    normal_flag = True # Merges the client and the server normally at the end
    immediate_flag = True # work in progress
    client_end = False # Used to check whether a client ends before another client
    client_end_enter_count = 0
    stream_iter = 0
    server_index = 0
    server_break_iter = 0
    client_break_iter = 0
    client_counter_temp = 0
    count_between_breakpoints_iter = 0
    num = 1
    counter = 0
    while(1):
        if i == 0:
            merge.append([[server_text[0], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(server_split(0, count_list[0]))
            uml.extend(socket)
            for z in range(0, count_list[0]):
                uml_insert(merge, counter, z, uml, start, end)
                merge[counter][z][1] = num
                num = num + 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(client_split(0, count_list1[0], client_counter))
            uml.extend(socket)
            for z in range(0, count_list1[0]):
                uml_insert(merge, counter, z, uml, start, end)
                merge[counter][z][1] = num
                num +=1
            counter +=1  
            
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            print(f'{i} 0')
            i = i + 1
            server_iter = server_iter + 1
            client_iter = client_iter + 1
        
        elif client_end_flag == 1:
            print('In client end flag')
            if client_end == True:
                for i in range(2):
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                    uml.extend(socket)
                    for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
#                     uml.extend(socket)
                    for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
#                 uml.extend(socket)
                for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_endpoints_index[stream_iter]-1], count_list[server_endpoints_index[stream_iter]]))
#                 uml.extend(socket)
                for z in range(count_list[server_endpoints_index[stream_iter]] - count_list[server_endpoints_index[stream_iter]-1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                client_length_count += 1
                client_sync_count1 -= 1
                if client_length_count == len(client_length1) and client_sync_count1 == 0:
                    merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
                    break
                continue
            server_iter = stream[stream_iter] + 1
            client_iter = stream[stream_iter] + client_sync_count1-1 + 1 + client_end_enter_count #not sure
            stream_iter = stream_iter + 1
            client_counter = client_counter_temp - 1
            client_counter_temp = client_counter_temp - 1
            client_iter
            print(f'{client_counter} CLIENT COUNTER before IN')
            for j in range(0, 2):
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
#                 uml.extend(socket)
                for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
#                 uml.extend(socket)
                for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter +=1
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                print("IN")
            print(f'{i}Value of i')
                        
            if immediate_flag == False:
                server_iter = server_break_iter + 1
                for i in range(count_between_breakpoints[0]):
                    print('In immediate false')
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
#                     uml.extend(socket)
                    for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
#                     uml.extend(socket)
                    for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                    
                
            client_sync_count1 = client_sync_count1 - 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            num += 1
            counter += 1
            print(f'{client_iter}--')
            merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
#             uml.extend(socket)
            for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                uml_insert(merge, counter, z, uml, start, end)
                merge[counter][z][1] = num
                num += 1
            counter += 1
            print("OUT")
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Call']])
            num += 1
            counter += 1
            if len(count_between_breakpoints) != 0:
                merge.append(server_split(count_list[server_endpoints[server_index]+count_between_breakpoints[0]], count_list[server_endpoints[server_index+1]]))
#                 uml.extend(socket)
                for z in range(count_list[server_endpoints[server_index+1]] - count_list[server_endpoints[server_index]+count_between_breakpoints[0]]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1 
            else:
                merge.append(server_split(count_list[server_endpoints[server_index]], count_list[server_endpoints[server_index+1]]))
#                 uml.extend(socket)
                for z in range(count_list[server_endpoints[server_index+1]] - count_list[server_endpoints[server_index]]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
            server_index = server_index + 1
            normal_flag = True
            if client_length_count == len(client_length1) and client_sync_count1 == 0:
                merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
#                 uml.extend(socket)
                for z in range(count_list[len(count_list) - 1] - count_list[len(count_list) - 2]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
            if client_sync_count1 == 0:
                merge_flag = 1
                client_end_flag = 0
                client_counter = client_length_count + 1
                print(f'{client_counter} client counter')
                print(f'{server_break_iter} server_break_iter')
                print(f'{client_break_iter} client_break_iter')
                server_iter = client_break_iter + 1
                client_iter = client_break_iter + 1
                print(f'{client_iter} client iter')
                print(f'{server_iter} server_iter')
                i = i + 1
                if client_length_count == len(client_length1):
                    break
                continue
                
        elif (i+1) % client_length1[client_counter - 1] == 0 or i == server_client_finish_point[client_counter - 1]:
            print('In elif.....client_length1[client_counter - 1]')
            print(f'{i}....Value of i In elif.....client_length1[client_counter - 1] ')
            if client_sync[client_counter - 1] == 1:
                print('In client_sync[client_counter - 1] == 1')
                server_break_iter = server_iter
                client_break_iter = client_iter
                client_counter_temp = client_counter
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for j in range(2):
                        merge.append([[server_text[0], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
#                         uml.extend(socket)
                        for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                            uml_insert(merge, counter, z, uml, start, end)
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[server_text[0], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
#                         uml.extend(socket)
                        for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                            uml_insert(merge, counter, z, uml, start, end)
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
#                     uml.extend(socket)
                    for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
#                     uml.extend(socket)
                    for z in range(count_list[server_endpoints_index[stream_iter]] - count_list[server_endpoints_index[stream_iter] - 1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    stream_iter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    client_counter  = client_counter_temp + 1 # not sure
                    client_counter_temp = client_counter #not sure
                    server_iter = server_startpoints[client_counter - 1]
                    client_iter = client_length[client_counter - 2]
                    client_end = False
                    merge_flag = 1
                    i = i + 1
                    client_length_count += 1
                    client_sync_count1 -= 1
                    if client_length_count == len(client_length) and client_sync_count1 == 0:
                        break
                    continue
                    
                if client_end_check[client_counter - 1] == 0:
                    client_end = True
                normal_flag = False
                server_iter = client_length[client_counter - 1] - k
                k = k + 1
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 1')
                client_iter = client_length[client_counter - 1]
                print(f'{client_iter} client_iter In client_sync[client_counter - 1] == 1')
                client_counter = client_counter + 1
                print(f'{i} value of i +')
                i = i + 1
                merge_flag = 1
                client_length_count += 1
                client_counter_temp = client_counter
                continue
            if normal_flag == True:
                print('In normal flag')
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
#                 uml.extend(socket)
                for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
#                 uml.extend(socket)
                for z in range(count_list[server_iter] - count_list[server_iter-1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                if client_counter == len(client_length):
                    merge.append(server_split(count_list[len(count_list)- 2], count_list[len(count_list) - 1]))
#                     uml.extend(socket)
                    for z in range(count_list[len(count_list) - 1] - count_list[len(count_list)- 2]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                client_length_count += 1
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                i = i + 1 
                if client_length_count == len(client_length1):
                    break
                merge_flag = 1
                client_counter +=1
                continue
                
            if client_sync[client_counter - 1] == 0:
                print('In client_sync[client_counter - 1] == 0')
                server_break_iter = server_iter
                client_break_iter = client_iter
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 0')
                print(f'{client_iter} client_iter in client_sync[client_counter - 1] == 0')
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for i in range(2):
                        merge.append([[server_text[0], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
#                         uml.extend(socket)
                        for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                            uml_insert(merge, counter, z, uml, start, end)
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[server_text[0], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
#                         uml.extend(socket)
                        for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                            uml_insert(merge, counter, z, uml, start, end)
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
#                     uml.extend(socket)
                    for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
#                     uml.extend(socket)
                    for z in range(count_list[server_endpoints_index[stream_iter]] - count_list[server_endpoints_index[stream_iter] - 1]):
                        uml_insert(merge, counter, z, uml, start, end)
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    stream_iter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    client_counter +=1
                    server_iter = server_break_iter
                    client_iter = client_break_iter
                    client_end_flag = 1
                    continue
                client_length_count += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
#                 uml.extend(socket)
                for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
#                 uml.extend(socket)
                for z in range(count_list[server_iter] - count_list[server_iter-1]):
                    uml_insert(merge, counter, z, uml, start, end)
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                client_end_flag = 1
                client_counter = client_counter - 1
                if len(count_between_breakpoints) > 0 and count_between_breakpoints[count_between_breakpoints_iter]:
                    immediate_flag = False
                    count_between_breakpoints_iter += 1
                
                
        elif merge_flag == 1:
            merge.append([[server_text[0], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
#             uml.extend(socket)
            for z in range(count_list[server_iter] - count_list[server_iter-1]):
                uml_insert(merge, counter, z, uml, start, end)
                merge[counter][z][1] = num
                num += 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(client_split(0 , count_list1[client_iter], client_counter))
#             uml.extend(socket)
            for z in range(0 , count_list1[client_iter]):
                uml_insert(merge, counter, z, uml, start, end)
                merge[counter][z][1] = num
                num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge_flag = 0
            print(f'{server_iter} server_iter in merge_flag')
            print(f'{client_iter} client_iter in merge_flag')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i} value of i in elif merge flag')
            i = i + 1
        else:
            merge.append([[server_text[0], num,' ','Method Call']])
            socket_insert_request(merge, counter - 1, socket)
            num += 1
            counter += 1
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
            uml.extend(socket)
            for z in range(count_list[server_iter] - count_list[server_iter-1]):
                uml_insert(merge, counter, z, uml, start, end)
                merge[counter][z][1] = num
                num += 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            socket_insert_response(merge, counter - 1, socket)
            num += 1
            counter += 1
            merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
            uml.extend(socket)
            for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                uml_insert(merge, counter, z, uml, start, end)
                merge[counter][z][1] = num 
                num += 1
            counter +=1
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            print(f'{server_iter} server_iter else')
            print(f'{client_iter} client_iter else')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i}else i')
            i = i + 1

0 0
1 server_iter else
1 client_iter else
1else i
2 server_iter else
2 client_iter else
2else i
In elif.....client_length1[client_counter - 1]
3....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
5 server_iter in client_sync[client_counter - 1] == 1
6 client_iter In client_sync[client_counter - 1] == 1
3 value of i +
5 server_iter in merge_flag
6 client_iter in merge_flag
4 value of i in elif merge flag
6 server_iter else
7 client_iter else
5else i
7 server_iter else
8 client_iter else
6else i
In elif.....client_length1[client_counter - 1]
7....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
10 server_iter in client_sync[client_counter - 1] == 1
12 client_iter In client_sync[client_counter - 1] == 1
7 value of i +
10 server_iter in merge_flag
12 client_iter in merge_flag
8 value of i in elif merge flag
11 server_iter else
13 client_iter else
9else i
12 server_iter else
14 client_iter e

In [321]:
print(socket)

['ClientA', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA', ':sends response', 'newline', 'ClientA', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA', ':sends response', 'newline', 'ClientA1', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA1', ':sends response', 'newline', 'ClientA1', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA1', ':sends response', 'newline', 'ClientA2', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA2', ':sends response', 'newline', 'ClientA2', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA2', ':sends response', 'newline', 'ClientA2', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA2', ':sends response', 'newline', 'ClientA2', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA2', ':sends response', 'newline']


In [304]:
#Used to serialize the execution trace
#Merges the execution traces of server and clients in the case where a new client communicates to the server during an
#existing client server communication
if sync == False:
    merge = []
    merge_flag = 0 #Used to start from a new client
    client_counter = 1
    client_sync_count1 = client_sync_count
    client_length_count = 0
    server_iter = 0
    client_iter = 0
    i = 0
    if len(count_between_breakpoints) != 0:
        k = 1 + count_between_breakpoints[0]
    else:
        k = 1
    client_end_flag = 0
    normal_flag = True # Merges the client and the server normally at the end
    immediate_flag = True # work in progress
    client_end = False # Used to check whether a client ends before another client
    client_end_enter_count = 0
    stream_iter = 0
    server_index = 0
    server_break_iter = 0
    client_break_iter = 0
    client_counter_temp = 0
    count_between_breakpoints_iter = 0
    num = 1
    counter = 0
    while(1):
        if i == 0:
            merge.append([[server_text[0], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(server_split(0, count_list[0]))
            for z in range(0, count_list[0]):
                merge[counter][z][1] = num
                num = num + 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(client_split(0, count_list1[0], client_counter))
            for z in range(0, count_list1[0]):
                merge[counter][z][1] = num
                num +=1
            counter +=1  
            
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            print(f'{i} 0')
            i = i + 1
            server_iter = server_iter + 1
            client_iter = client_iter + 1
        
        elif client_end_flag == 1:
            print('In client end flag')
            if client_end == True:
                for i in range(2):
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                    for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                    for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_endpoints_index[stream_iter]-1], count_list[server_endpoints_index[stream_iter]]))
                for z in range(count_list[server_endpoints_index[stream_iter]] - count_list[server_endpoints_index[stream_iter]-1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                client_length_count += 1
                client_sync_count1 -= 1
                if client_length_count == len(client_length1) and client_sync_count1 == 0:
                    merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
                    break
                continue
            server_iter = stream[stream_iter] + 1
            client_iter = stream[stream_iter] + client_sync_count1-1 + 1 + client_end_enter_count #not sure
            stream_iter = stream_iter + 1
            client_counter = client_counter_temp - 1
            client_counter_temp = client_counter_temp - 1
            client_iter
            print(f'{client_counter} CLIENT COUNTER before IN')
            for j in range(0, 2):
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter +=1
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                print("IN")
            print(f'{i}Value of i')
                        
            if immediate_flag == False:
                server_iter = server_break_iter + 1
                for i in range(count_between_breakpoints[0]):
                    print('In immediate false')
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                    for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                    for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                    
                
            client_sync_count1 = client_sync_count1 - 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            num += 1
            counter += 1
            print(f'{client_iter}--')
            merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
            for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                merge[counter][z][1] = num
                num += 1
            counter += 1
            print("OUT")
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Call']])
            num += 1
            counter += 1
            if len(count_between_breakpoints) != 0:
                merge.append(server_split(count_list[server_endpoints[server_index]+count_between_breakpoints[0]], count_list[server_endpoints[server_index+1]]))
                for z in range(count_list[server_endpoints[server_index+1]] - count_list[server_endpoints[server_index]+count_between_breakpoints[0]]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1 
            else:
                merge.append(server_split(count_list[server_endpoints[server_index]], count_list[server_endpoints[server_index+1]]))
                for z in range(count_list[server_endpoints[server_index+1]] - count_list[server_endpoints[server_index]]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
            server_index = server_index + 1
            normal_flag = True
            if client_length_count == len(client_length1) and client_sync_count1 == 0:
                merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
                for z in range(count_list[len(count_list) - 1] - count_list[len(count_list) - 2]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
            if client_sync_count1 == 0:
                merge_flag = 1
                client_end_flag = 0
                client_counter = client_length_count + 1
                print(f'{client_counter} client counter')
                print(f'{server_break_iter} server_break_iter')
                print(f'{client_break_iter} client_break_iter')
                server_iter = client_break_iter + 1
                client_iter = client_break_iter + 1
                print(f'{client_iter} client iter')
                print(f'{server_iter} server_iter')
                i = i + 1
                if client_length_count == len(client_length1):
                    break
                continue
                
        elif (i+1) % client_length1[client_counter - 1] == 0 or i == server_client_finish_point[client_counter - 1]:
            print('In elif.....client_length1[client_counter - 1]')
            print(f'{i}....Value of i In elif.....client_length1[client_counter - 1] ')
            if client_sync[client_counter - 1] == 1:
                print('In client_sync[client_counter - 1] == 1')
                server_break_iter = server_iter
                client_break_iter = client_iter
                client_counter_temp = client_counter
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for j in range(2):
                        merge.append([[server_text[0], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                        for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[server_text[0], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                        for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                    for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
                    for z in range(count_list[server_endpoints_index[stream_iter]] - count_list[server_endpoints_index[stream_iter] - 1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    stream_iter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    client_counter  = client_counter_temp + 1 # not sure
                    client_counter_temp = client_counter #not sure
                    server_iter = server_startpoints[client_counter - 1]
                    client_iter = client_length[client_counter - 2]
                    client_end = False
                    merge_flag = 1
                    i = i + 1
                    client_length_count += 1
                    client_sync_count1 -= 1
                    if client_length_count == len(client_length) and client_sync_count1 == 0:
                        break
                    continue
                    
                if client_end_check[client_counter - 1] == 0:
                    client_end = True
                normal_flag = False
                server_iter = client_length[client_counter - 1] - k
                k = k + 1
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 1')
                client_iter = client_length[client_counter - 1]
                print(f'{client_iter} client_iter In client_sync[client_counter - 1] == 1')
                client_counter = client_counter + 1
                print(f'{i} value of i +')
                i = i + 1
                merge_flag = 1
                client_length_count += 1
                client_counter_temp = client_counter
                continue
            if normal_flag == True:
                print('In normal flag')
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
                for z in range(count_list[server_iter] - count_list[server_iter-1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                if client_counter == len(client_length):
                    merge.append(server_split(count_list[len(count_list)- 2], count_list[len(count_list) - 1]))
                    for z in range(count_list[len(count_list) - 1] - count_list[len(count_list)- 2]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                client_length_count += 1
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                i = i + 1 
                if client_length_count == len(client_length1):
                    break
                merge_flag = 1
                client_counter +=1
                continue
                
            if client_sync[client_counter - 1] == 0:
                print('In client_sync[client_counter - 1] == 0')
                server_break_iter = server_iter
                client_break_iter = client_iter
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 0')
                print(f'{client_iter} client_iter in client_sync[client_counter - 1] == 0')
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for i in range(2):
                        merge.append([[server_text[0], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                        for z in range(count_list[server_iter] - count_list[server_iter - 1]):
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[server_text[0], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                        num += 1
                        counter += 1
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                        for z in range(count_list1[client_iter] - count_list1[client_iter - 1]):
                            merge[counter][z][1] = num
                            num += 1
                        counter += 1
                        merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                        num += 1
                        counter += 1
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                    for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    merge.append([[server_text[0], num,' ','Method Call']])
                    num += 1
                    counter += 1
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
                    for z in range(count_list[server_endpoints_index[stream_iter]] - count_list[server_endpoints_index[stream_iter] - 1]):
                        merge[counter][z][1] = num
                        num += 1
                    counter += 1
                    stream_iter += 1
                    merge.append([[server_text[0], num,' ','Method Exit']])
                    num += 1
                    counter += 1
                    client_counter +=1
                    server_iter = server_break_iter
                    client_iter = client_break_iter
                    client_end_flag = 1
                    continue
                client_length_count += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
                num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Call']])
                num += 1
                counter += 1
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
                for z in range(count_list[server_iter] - count_list[server_iter-1]):
                    merge[counter][z][1] = num
                    num += 1
                counter += 1
                merge.append([[server_text[0], num,' ','Method Exit']])
                num += 1
                counter += 1
                client_end_flag = 1
                client_counter = client_counter - 1
                if len(count_between_breakpoints) > 0 and count_between_breakpoints[count_between_breakpoints_iter]:
                    immediate_flag = False
                    count_between_breakpoints_iter += 1
                
                
        elif merge_flag == 1:
            merge.append([[server_text[0], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
            for z in range(count_list[server_iter] - count_list[server_iter-1]):
                merge[counter][z][1] = num
                num += 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(client_split(0 , count_list1[client_iter], client_counter))
            for z in range(0 , count_list1[client_iter]):
                merge[counter][z][1] = num
                num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge_flag = 0
            print(f'{server_iter} server_iter in merge_flag')
            print(f'{client_iter} client_iter in merge_flag')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i} value of i in elif merge flag')
            i = i + 1
        else:
            merge.append([[server_text[0], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
            for z in range(count_list[server_iter] - count_list[server_iter-1]):
                merge[counter][z][1] = num
                num += 1
            counter += 1
            merge.append([[server_text[0], num,' ','Method Exit']])
            num += 1
            counter += 1
            merge.append([[client_text[client_counter - 1], num,' ','Method Call']])
            num += 1
            counter += 1
            merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
            for z in range(count_list1[client_iter] - count_list1[client_iter-1]):
                merge[counter][z][1] = num 
                num += 1
            counter +=1
            merge.append([[client_text[client_counter - 1], num,' ','Method Exit']])
            num += 1
            counter += 1
            print(f'{server_iter} server_iter else')
            print(f'{client_iter} client_iter else')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i}else i')
            i = i + 1

0 0
1 server_iter else
1 client_iter else
1else i
2 server_iter else
2 client_iter else
2else i
In elif.....client_length1[client_counter - 1]
3....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
5 server_iter in client_sync[client_counter - 1] == 1
6 client_iter In client_sync[client_counter - 1] == 1
3 value of i +
5 server_iter in merge_flag
6 client_iter in merge_flag
4 value of i in elif merge flag
6 server_iter else
7 client_iter else
5else i
7 server_iter else
8 client_iter else
6else i
In elif.....client_length1[client_counter - 1]
7....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
10 server_iter in client_sync[client_counter - 1] == 1
12 client_iter In client_sync[client_counter - 1] == 1
7 value of i +
10 server_iter in merge_flag
12 client_iter in merge_flag
8 value of i in elif merge flag
11 server_iter else
13 client_iter else
9else i
12 server_iter else
14 client_iter e

In [322]:
print(merge[4])

[['SYSTEM', 21, ' "SYSTEM"', ' "System Start"', ' " "'], ['main', 22, ' "SYSTEM"', ' "Thread Start"', ' " main (1)"'], ['main', 23, ' "SYSTEM"', ' "Type Load"', ' " class=java.lang.Object"'], ['main', 24, ' "SYSTEM"', ' "Type Load"', ' " class=Client.ClientA"'], ['main', 25, ' "Client/ClientA.java:18"', ' "Method Call"', ' " caller=SYSTEM', ' target=Client.ClientA#main:1"'], ['main', 26, ' "Client/ClientA.java:14"', ' "Method Entered"', ' " "'], ['main', 27, ' "SYSTEM"', ' "Type Load"', ' " class=java.lang.String"'], ['main', 28, ' "Client/ClientA.java:14"', ' "New Object"', ' " object=java.lang.String[]:1"'], ['main', 29, ' "Client/ClientA.java:14"', ' "Variable Write"', ' " context=Client.ClientA#main:1', ' args=java.lang.String[]:1"'], ['main', 30, ' "Client/ClientA.java:18"', ' "Line Step"', ' " file=Client/ClientA.java', ' line=18"'], ['main', 31, ' "Client/ClientA.java:18"', ' "Variable Write"', ' " context=Client.ClientA#main:1', ' scn=java.util.Scanner (id=44)"'], ['main', 32, 

In [306]:
#Code for generating UML diagram
#Merges the execution traces of server and clients in the case where a new client communicates to the server during an
#existing client server communication
if sync == False:
    uml = ['@startuml']
    start = []
    end = []
    socket = []
    merge = []
    merge_flag = 0 #Used to start from a new client
    client_counter = 1
    client_sync_count1 = client_sync_count
    client_length_count = 0
    server_iter = 0
    client_iter = 0
    i = 0
    if len(count_between_breakpoints) != 0:
        k = 1 + count_between_breakpoints[0]
    else:
        k = 1
    client_end_flag = 0
    normal_flag = True # Merges the client and the server normally at the end
    immediate_flag = True # work in progress
    client_end = False # Used to check whether a client ends before another client
    client_end_enter_count = 0
    stream_iter = 0
    server_index = 0
    server_break_iter = 0
    client_break_iter = 0
    client_counter_temp = 0
    count_between_breakpoints_iter = 0
    while(1):
        if i == 0:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(0, count_list[0]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(0, count_list1[0], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 4,' ','Method Exit']])
            print(f'{i} 0')
            i = i + 1
            server_iter = server_iter + 1
            client_iter = client_iter + 1
        
        elif client_end_flag == 1:
            print('In client end flag')
            if client_end == True:
                for i in range(2):
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                    merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                    merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_endpoints_index[stream_iter]-1], count_list[server_endpoints_index[stream_iter]]))
                merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                client_length_count += 1
                client_sync_count1 -= 1
                if client_length_count == len(client_length1) and client_sync_count1 == 0:
                    merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
                    break
                continue
            server_iter = stream[stream_iter] + 1
            client_iter = stream[stream_iter] + client_sync_count1-1 + 1 + client_end_enter_count #not sure
            stream_iter = stream_iter + 1
            client_counter = client_counter_temp - 1
            client_counter_temp = client_counter_temp - 1
            client_iter
            print(f'{client_counter} CLIENT COUNTER before IN')
            for j in range(0, 2):
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                print("IN")
            print(f'{i}Value of i')
                        
            if immediate_flag == False:
                server_iter = server_break_iter + 1
                for i in range(count_between_breakpoints[0]):
                    print('In immediate false')
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                    merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                    merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], count_list1[0] + 10,' ','Method Exit']])
                    server_iter = server_iter + 1
                    client_iter = client_iter + 1
                    
                
            client_sync_count1 = client_sync_count1 - 1
            merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
            print(f'{client_iter}--')
            merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
            print("OUT")
            merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
            merge.append([[server_text[0], 1,' ','Method Call']])
            if len(count_between_breakpoints) != 0:
                merge.append(server_split(count_list[server_endpoints[server_index]+count_between_breakpoints[0]], count_list[server_endpoints[server_index+1]]))
            else:
                merge.append(server_split(count_list[server_endpoints[server_index]], count_list[server_endpoints[server_index+1]]))
                
            server_index = server_index + 1
            normal_flag = True
            if client_length_count == len(client_length1) and client_sync_count1 == 0:
                merge.append(server_split(count_list[len(count_list) - 2], count_list[len(count_list) - 1]))
            if client_sync_count1 == 0:
                merge_flag = 1
                client_end_flag = 0
                client_counter = client_length_count + 1
                print(f'{client_counter} client counter')
                print(f'{server_break_iter} server_break_iter')
                print(f'{client_break_iter} client_break_iter')
                server_iter = client_break_iter + 1
                client_iter = client_break_iter + 1
                print(f'{client_iter} client iter')
                print(f'{server_iter} server_iter')
                i = i + 1
                if client_length_count == len(client_length1):
                    break
                continue
                
        elif (i+1) % client_length1[client_counter - 1] == 0 or i == server_client_finish_point[client_counter - 1]:
            print('In elif.....client_length1[client_counter - 1]')
            print(f'{i}....Value of i In elif.....client_length1[client_counter - 1] ')
            if client_sync[client_counter - 1] == 1:
                print('In client_sync[client_counter - 1] == 1')
                server_break_iter = server_iter
                client_break_iter = client_iter
                client_counter_temp = client_counter
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for j in range(2):
                        merge.append([[server_text[0], 1,' ','Method Call']])
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                        merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                        merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                        merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
                    stream_iter += 1
                    merge.append([[server_text[0], 1,' ','Method Exit']])
                    client_counter  = client_counter_temp + 1 # not sure
                    client_counter_temp = client_counter #not sure
                    server_iter = server_startpoints[client_counter - 1]
                    client_iter = client_length[client_counter - 2]
                    client_end = False
                    merge_flag = 1
                    i = i + 1
                    client_length_count += 1
                    client_sync_count1 -= 1
                    if client_length_count == len(client_length) and client_sync_count1 == 0:
                        break
                    continue
                    
                if client_end_check[client_counter - 1] == 0:
                    client_end = True
                normal_flag = False
                server_iter = client_length[client_counter - 1] - k
                k = k + 1
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 1')
                client_iter = client_length[client_counter - 1]
                print(f'{client_iter} client_iter In client_sync[client_counter - 1] == 1')
                client_counter = client_counter + 1
                print(f'{i} value of i +')
                i = i + 1
                merge_flag = 1
                client_length_count += 1
                client_counter_temp = client_counter
                continue
            if normal_flag == True:
                print('In normal flag')
                merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
                if client_counter == len(client_length):
                    merge.append(server_split(count_list[len(count_list)- 2], count_list[len(count_list) - 1]))
                merge.append([[server_text[0], 1,' ','Method Exit']])
                client_length_count += 1
                server_iter = server_iter + 1
                client_iter = client_iter + 1
                i = i + 1 
                if client_length_count == len(client_length1):
                    break
                merge_flag = 1
                client_counter +=1
                continue
                
            if client_sync[client_counter - 1] == 0:
                print('In client_sync[client_counter - 1] == 0')
                server_break_iter = server_iter
                client_break_iter = client_iter
                print(f'{server_iter} server_iter in client_sync[client_counter - 1] == 0')
                print(f'{client_iter} client_iter in client_sync[client_counter - 1] == 0')
                if client_end == True:
                    print(f'In client_end ')
                    client_end_enter_count += 1
                    server_iter = stream[stream_iter] + 1
                    client_iter = stream[stream_iter] + 1
                    client_counter -= 1
                    for i in range(2):
                        merge.append([[server_text[0], 1,' ','Method Call']])
                        merge.append(server_split(count_list[server_iter - 1], count_list[server_iter]))
                        merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
                        merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
                        merge.append(client_split(count_list1[client_iter - 1], count_list1[client_iter], client_counter))
                        merge.append([[client_text[client_counter - 1], count_list1[0] + 13,' ','Method Exit']])
                        server_iter = server_iter + 1
                        client_iter = client_iter + 1
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                    merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                    merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                    merge.append([[server_text[0], 1,' ','Method Call']])
                    merge.append(server_split(count_list[server_endpoints_index[stream_iter] - 1], count_list[server_endpoints_index[stream_iter]]))
                    stream_iter += 1
                    merge.append([[server_text[0], 1,' ','Method Exit']])
                    client_counter +=1
                    server_iter = server_break_iter
                    client_iter = client_break_iter
                    client_end_flag = 1
                    continue
                client_length_count += 1
                merge.append([[client_text[client_counter - 1], 1,' ','Method Call']])
                merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
                merge.append([[client_text[client_counter - 1], 1,' ','Method Exit']])
                merge.append([[server_text[0], 1,' ','Method Call']])
                merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
                merge.append([[server_text[0], 1,' ','Method Exit']])
                client_end_flag = 1
                client_counter = client_counter - 1
                if len(count_between_breakpoints) > 0 and count_between_breakpoints[count_between_breakpoints_iter] > 0:
                    immediate_flag = False
                    count_between_breakpoints_iter += 1
                
                
        elif merge_flag == 1:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(0 , count_list1[client_iter], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 4,' ','Method Exit']])
            merge_flag = 0
            print(f'{server_iter} server_iter in merge_flag')
            print(f'{client_iter} client_iter in merge_flag')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i} value of i in elif merge flag')
            i = i + 1
        else:
            merge.append([[server_text[0], 1,' ','Method Call']])
            merge.append(server_split(count_list[server_iter-1], count_list[server_iter]))
            merge.append([[server_text[0], count_list[0] + 2,' ','Method Exit']])
            merge.append([[client_text[client_counter - 1], count_list[0] + 3,' ','Method Call']])
            merge.append(client_split(count_list1[client_iter-1], count_list1[client_iter], client_counter))
            merge.append([[client_text[client_counter - 1], count_list1[0] + 10,' ','Method Exit']])
            print(f'{server_iter} server_iter else')
            print(f'{client_iter} client_iter else')
            server_iter = server_iter + 1
            client_iter = client_iter + 1
            print(f'{i}else i')
            i = i + 1

0 0
1 server_iter else
1 client_iter else
1else i
2 server_iter else
2 client_iter else
2else i
In elif.....client_length1[client_counter - 1]
3....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
5 server_iter in client_sync[client_counter - 1] == 1
6 client_iter In client_sync[client_counter - 1] == 1
3 value of i +
5 server_iter in merge_flag
6 client_iter in merge_flag
4 value of i in elif merge flag
6 server_iter else
7 client_iter else
5else i
7 server_iter else
8 client_iter else
6else i
In elif.....client_length1[client_counter - 1]
7....Value of i In elif.....client_length1[client_counter - 1] 
In client_sync[client_counter - 1] == 1
10 server_iter in client_sync[client_counter - 1] == 1
12 client_iter In client_sync[client_counter - 1] == 1
7 value of i +
10 server_iter in merge_flag
12 client_iter in merge_flag
8 value of i in elif merge flag
11 server_iter else
13 client_iter else
9else i
12 server_iter else
14 client_iter e

In [323]:
with open("test.csv", "w", newline="") as f:
    writer = csv.writer(f)
    for i in range(len(merge)):
        writer.writerows(merge[i])

In [188]:
num = 2
k = 1
temp = 0
client_end_counter = 0
client_start_counter = 0
if sync == True:
    while temp < client_length[len(client_length) - 1]:
    #     if k == len(merge) - 1:
    #         break
        if (temp + 1)%client_length[client_end_counter] == 0:
            print(f'{k} value of k')
            for n in range(count_list1[temp] - count_list1[temp - 1]):
                merge[k][n][1] = num
                num = num + 1
            k += 1
            print(k)
    #         if k == len(merge) - 1:
    #             break
            for o in range(2):
                z = 0
                merge[k][z][1] = num
                num = num + 1
                z += 1
                k += 1
            print(k)
    #         if k == len(merge) - 1:
    #             break
            for j in range(count_list[temp] - count_list[temp - 1]):
                merge[k][j][1] = num
                num = num + 1
            k += 1
            print(k)
    #         if k == len(merge) - 1:
    #             break

            if (client_start_counter+1) == len(client_length):
                print(f'{k} +++')
                for i in range(len(merge[k])):
                    merge[k][i][1] = num
                    num = num + 1
            m = 0    
            for m in range(2):
                merge[k][0][1] = num
                num = num + 1
                k += 1

            print(k)
    #         if k == len(merge) - 1:
    #             break
            temp += 1
            client_end_counter +=1
    #         if k == len(merge) - 1:
    #             break
            print(f'{temp}+')

            continue


        #Server
        if temp == 0:
            for j in range(count_list[temp]):
                merge[k][j][1] = num
                num = num + 1
            k += 1
            print(k)
        else:
            for j in range(count_list[temp] - count_list[temp - 1]):
                merge[k][j][1] = num
                num = num + 1
            k += 1
            print(k)
    #     if k == len(merge) - 1:
    #         break
        #server exit client call    
        m = 0    
        for m in range(2):
            merge[k][0][1] = num
            num = num + 1
            k += 1

        print(k)

        #Client

    #     print(temp)
        if temp == 0 or temp%client_length[client_start_counter] == 0:
            print(f'{k}cli')
            print(f'{client_start_counter}cli_Counter')
            for n in range(count_list1[temp]):
                merge[k][n][1] = num
                num = num + 1
            k += 1
            if temp != 0:
                client_start_counter +=1
            print(k)
    #     elif temp == len(count_list)-2:
    #         for n in range(count_list1[temp + 1] - count_list1[temp]):
    #             merge[k][n][1] = num
    #             num = num + 1
    #         k += 1
    #         print(k)
        else:
            for n in range(count_list1[temp] - count_list1[temp - 1]):
                merge[k][n][1] = num
                num = num + 1
            k += 1
            print(k)
        if k == len(merge) - 1:
            break
        #client exit server call   
        for o in range(2):
            z = 0
            merge[k][z][1] = num
            num = num + 1
            z += 1
            k += 1
        print(k)
        temp += 1
        print(f'{temp}+')



In [324]:
print(client_length)
print(len(merge))
# print(client_length[len(client_length) - 1])

[6, 12, 18]
107


In [325]:
merge[len(merge) - 1][0][1] = num

In [326]:
with open("merged-csv.csv", "w", newline="") as f:
    writer = csv.writer(f)
    for i in range(len(merge)):
        writer.writerows(merge[i])

In [327]:
print(merge[64])

[['main', 293, ' "ClientA2.java:35"', ' "Line Step"', ' " file=ClientA2.java', ' line=35"'], ['main', 294, ' "ClientA2.java:36"', ' "Line Step"', ' " file=ClientA2.java', ' line=36"'], ['main', 295, ' "ClientA2.java:36"', ' "Variable Write"', ' " context=ClientA2#main:1', ' tosend=""Exit"""'], ['main', 296, ' "ClientA2.java:37"', ' "Line Step"', ' " file=ClientA2.java', ' line=37"'], ['main', 297, ' "ClientA2.java:37"', ' "Variable Write"', ' " context=ClientA2#main:1', ' dos=java.io.DataOutputStream (id=61)"']]


In [328]:
temp = 2
print(merge[1][10][3])

 "Variable Write"


In [130]:
#New uml generating algorithm
#Code for generating uml diagram
if sync == True:
    temp = 0
    k = 1
    x = 0
    uml = ["@startuml"]
    start = []
    end = []
    socket = []
    client_counter = 0
    while 1:
        if k == len(merge) - 1:
            break

        if (temp+1)%client_length[client_counter] == 0:
            print(f'{k}-----')
            print(f'{client_counter} +-')
            print(f'{temp} value of temp')

            for j in range(count_list1[temp] - count_list1[temp-1]):
                if merge[k][j][3] == ' "Method Call"':
                    s = merge[k][j][4]
                    flg = 0
                    for c in range(len(s)):
                        if s[c] == '#':
                            tem = c
                        if s[c] == '.':
                            tem1 = c
                            flg = 1
                    if flg == 0:
                        s = s[10:tem+1]
                    else:
                        s = s[tem1+1:tem]
                    s = s.replace(":",".")
                    uml.append(s)
                    start.append(s)
                    uml.append("->")
                    s1 = merge[k][j][5]
                    flg1 = 0
                    for c in range(len(s1)):
                        if s1[c] == '#':
                            tem = c
                        if s1[c] == '.':
                            tem1 = c
                            flg1 = 1
                    if flg1 == 0:
                        message = s1[tem+1:-1]
                        s1 = s1[8:tem]
                    else:
                        message = s1[tem+1:-1]
                        s1 = s1[tem1+1:tem]
                    s1 = s1.replace(":",".")
                    uml.append(s1)
                    uml.append(":")
                    uml.append(message)
                    uml.append("newline")
                    uml.append(f'activate {s1}')
                    uml.append("newline")
                if merge[k][j][3] == ' "Method Exit"':
                    s2 = merge[k][j][4]
                    flg2 = 0
                    for c in range(len(s2)):
                        if s2[c] == '#':
                            tem = c
                        if s2[c] == '.':
                            tem1 = c
                            flg2 = 1
                    if flg2 == 0:
                        s2 = s2[12:tem]
                    else:
                        s2 = s2[tem1+1:tem]
                    s2 = s2.replace(":",".")
                    end.append(s2)
                    uml.append(end[len(end) - 1])
                    uml.append("-->")
                    uml.append(start[len(start) - 1])
                    start.remove(start[len(start) - 1])
                    uml.append("newline")
                    uml.append(f'deactivate {end[len(end) - 1]}')
                    uml.append("newline")
            client_counter = client_counter + 1
            k+=3
            print(f'{k}----')
            for j in range(count_list[temp] - count_list[temp - 1]):
                if merge[k][j][3] == ' "Method Call"':
                    s = merge[k][j][4]
                    if s == ' " caller=Ljava/util/concurrent/ThreadPoolExecutor$Worker;.run()' or s == ' " caller=Ljava/util/concurrent/ThreadPoolExecutor;.runWorker(Ljava/util/concurrent/ThreadPoolExecutor$Worker;)':
                        continue
                    s1 = merge[k][j][5]
                    if s1 == ' target=Ljava/util/concurrent/ThreadPoolExecutor$Worker;.run()"':
                        continue
                    print(s)
                    s = s.replace(":",".")
                    for c in range(len(s)):
                        if s[c] == '#':
                            tem = c
                    s = s[10:tem]
                    uml.append(s)
                    start.append(s)
                    uml.append("->")
                    s1 = s1.replace(":",".")
                    for c in range(len(s1)):
                        if s1[c] == '#':
                            tem = c
                    message = s1[tem+1:-1]
                    s1 = s1[8:tem]
                    uml.append(s1)
                    uml.append(":")
                    uml.append(message)
                    uml.append("newline")
                    uml.append(f'activate {s1}')
                    uml.append("newline")
                if merge[k][j][3] == ' "Method Exit"':
                    s2 = merge[k][j][4]
                    for c in range(len(s2)):
                        if s2[c] == '#':
                            tem = c
                    s2 = s2[12:tem]
                    s2 = s2.replace(":",".")
                    end.append(s2)
                    uml.append(end[len(end) - 1])
                    uml.append("-->")
                    uml.append(start[len(start) - 1])
                    start.remove(start[len(start) - 1])
                    uml.append('newline')
                    uml.append(f'deactivate {end[len(end) - 1]}')
                    uml.append("newline")
            k += 1

            if client_counter == len(client_length):
                print(f'{k} +++')
                for i in range(len(merge[k])):
                    merge[k][i][1] = num
                    num = num + 1
                break

            for i in range(2):
                k+=1
            temp+=1
            print(f'{temp}+')
            continue

        #Server
        if temp == 0:
            uml.extend(socket)
            for j in range(count_list[temp]):
                if merge[k][j][3] == ' "Method Call"':
                    s = merge[k][j][4]
                    if s == ' " caller=Ljava/util/concurrent/ThreadPoolExecutor$Worker;.run()' or s == ' " caller=Ljava/util/concurrent/ThreadPoolExecutor;.runWorker(Ljava/util/concurrent/ThreadPoolExecutor$Worker;)':
                        continue
                    s1 = merge[k][j][5]
                    if s1 == ' target=Ljava/util/concurrent/ThreadPoolExecutor$Worker;.run()"':
                        continue
                    s = s.replace(":",".")
                    for c in range(len(s)):
                        if s[c] == '#':
                            tem = c
                    s = s[10:tem]
                    uml.append(s)
                    start.append(s)
                    uml.append("->")
                    s1 = s1.replace(":",".")
                    for c in range(len(s1)):
                        if s1[c] == '#':
                            tem = c
                    message = s1[tem+1:-1]
                    s1 = s1[8:tem]
                    uml.append(s1)
                    uml.append(":")
                    uml.append(message)
                    uml.append("newline")
                    uml.append(f'activate {s1}')
                    uml.append("newline")
                if merge[k][j][3] == ' "Method Exit"':
                    s2 = merge[k][j][4]
                    for c in range(len(s2)):
                        if s2[c] == '#':
                            tem = c
                    s2 = s2[12:tem]
                    s2 = s2.replace(":",".")
                    end.append(s2)
                    uml.append(end[len(end) - 1])
                    uml.append("-->")
                    uml.append(start[len(start) - 1])
                    start.remove(start[len(start) - 1])
                    uml.append("newline")
                    uml.append(f'deactivate {end[len(end) - 1]}')
                    uml.append("newline")

            k += 1
        else:
            uml.extend(socket)
            for j in range(count_list[temp] - count_list[temp - 1]):
                if merge[k][j][3] == ' "Method Call"':
                    s = merge[k][j][4]
                    if s == ' " caller=Ljava/util/concurrent/ThreadPoolExecutor$Worker;.run()' or s == ' " caller=Ljava/util/concurrent/ThreadPoolExecutor;.runWorker(Ljava/util/concurrent/ThreadPoolExecutor$Worker;)':
                        continue
                    s1 = merge[k][j][5]
                    if s1 == ' target=Ljava/util/concurrent/ThreadPoolExecutor$Worker;.run()"':
                        continue
                    print(s)
                    s = s.replace(":",".")
                    for c in range(len(s)):
                        if s[c] == '#':
                            tem = c
                    s = s[10:tem]
                    uml.append(s)
                    start.append(s)
                    uml.append("->")
                    s1 = s1.replace(":",".")
                    for c in range(len(s1)):
                        if s1[c] == '#':
                            tem = c
                    message = s1[tem+1:-1]
                    s1 = s1[8:tem]
                    uml.append(s1)
                    uml.append(":")
                    uml.append(message)
                    uml.append("newline")
                    uml.append(f'activate {s1}')
                    uml.append("newline")
                if merge[k][j][3] == ' "Method Exit"':
                    s2 = merge[k][j][4]
                    for c in range(len(s2)):
                        if s2[c] == '#':
                            tem = c
                    s2 = s2[12:tem]
                    s2 = s2.replace(":",".")
                    end.append(s2)
                    uml.append(end[len(end) - 1])
                    uml.append("-->")
                    uml.append(start[len(start) - 1])
                    start.remove(start[len(start) - 1])
                    uml.append("newline")
                    uml.append(f'deactivate {end[len(end) - 1]}')
                    uml.append("newline")
            k += 1
        if temp == 0:
            print(uml)
        if k == len(merge) - 1:
            break
        socket = []
        #server exit client call     
        for m in range(2):
            if temp != 0 and temp%client_length[client_counter - 1] != 0:
                socket.append(merge[k][0][0])
                print(f'Merge list {merge[k][0][0]}')
                if m == 0:
                    socket.append("-->")
                else:
                    socket.append(":sends response")
                    socket.append("newline")
            k += 1

        #Client
        if temp == 0 or temp%client_length[client_counter - 1] == 0:
            uml.extend(socket)
            for j in range(count_list1[temp]):
                if merge[k][j][3] == ' "Method Call"':
                    s = merge[k][j][4]
                    flg = 0
                    for c in range(len(s)):
                        if s[c] == '#':
                            tem = c
                        if s[c] == '.':
                            tem1 = c
                            flg = 1
                    if flg == 0:
                        s = s[10:tem+1]
                    else:
                        s = s[tem1+1:tem]
                    s = s.replace(":",".")
                    uml.append(s)
                    start.append(s)
                    uml.append("->")
                    s1 = merge[k][j][5]
                    flg1 = 0
                    for c in range(len(s1)):
                        if s1[c] == '#':
                            tem = c
                        if s1[c] == '.':
                            tem1 = c
                            flg1 = 1
                    if flg1 == 0:
                        message = s1[tem+1 : -1]
                        s1 = s1[8:tem]
                    else:
                        message = s1[tem + 1 : -1]
                        s1 = s1[tem1+1:tem]
                    s1 = s1.replace(":",".")
                    uml.append(s1)
                    uml.append(":")
                    uml.append(message)
                    uml.append("newline")
                    uml.append(f'activate {s1}')
                    uml.append("newline")
                if merge[k][j][3] == ' "Method Exit"':
                    s2 = merge[k][j][4]
                    flg2 = 0
                    for c in range(len(s2)):
                        if s2[c] == '#':
                            tem = c
                        if s2[c] == '.':
                            tem1 = c
                            flg2 = 1
                    if flg2 == 0:
                        s2 = s2[12:tem]
                    else:
                        s2 = s2[tem1+1:tem]
                    s2 = s2.replace(":",".")
                    end.append(s2)
                    uml.append(end[len(end) - 1])
                    uml.append("-->")
                    uml.append(start[len(start) - 1])
                    start.remove(start[len(start) - 1])
                    uml.append("newline")
                    uml.append(f'deactivate {end[len(end) - 1]}')
                    uml.append('newline')
                    client_counter = client_counter + 1

            k += 1
        elif temp == len(count_list)-2:
            uml.extend(socket)
            for j in range(count_list1[temp + 1] - count_list1[temp]):
                if merge[k][j][3] == ' "Method Call"':
                    s = merge[k][j][4]
                    flg = 0
                    for c in range(len(s)):
                        if s[c] == '#':
                            tem = c
                        if s[c] == '.':
                            tem1 = c
                            flg = 1
                    if flg == 0:
                        s = s[10:tem]
                    else:
                        s = s[tem1+1:tem]
                    s = s.replace(":",".")
                    uml.append(s)
                    start.append(s)
                    uml.append("->")
                    s1 = merge[k][j][5]
                    s1 = s1.replace(":",".")
                    flg1 = 0
                    for c in range(len(s1)):
                        if s1[c] == '#':
                            tem = c
                        if s1[c] == '.':
                            tem1 = c
                            flg1 = 1
                    if flg1 == 0:
                        message = s1[tem+1:-1]
                        s1 = s1[8:tem]
                    else:
                        message = s1[tem+1:-1]
                        s1 = s1[tem1+1:tem]
                    uml.append(s1)
                    uml.append(":")
                    uml.append(message)
                    uml.append("newline")
                    uml.append(f'activate {s1}')
                    uml.append("newline")
                if merge[k][j][3] == ' "Method Exit"':
                    s2 = merge[k][j][4]
                    flg2 = 0
                    for c in range(len(s2)):
                        if s2[c] == '#':
                            tem = c
                        if s2[c] == '.':
                            tem1 = c
                            flg2 = 1
                    if flg2 == 0:
                        s2 = s2[12:tem]
                    else:
                        s2 = s2[tem1+1:tem]
                    s2 = s2.replace(":",".")
                    end.append(s2)
                    uml.append(end[len(end) - 1])
                    uml.append("-->")
                    uml.append(start[len(start) - 1])
                    start.remove(start[len(start) - 1])
                    uml.append("newline")
                    uml.append(f'deactivate {end[len(end) - 1]}')
                    uml.append("newline")
            k += 1
        else:
            uml.extend(socket)
            for j in range(count_list1[temp] - count_list1[temp - 1]):
                if merge[k][j][3] == ' "Method Call"':
                    s = merge[k][j][4]
                    flg = 0
                    for c in range(len(s)):
                        if s[c] == '#':
                            tem = c
                        if s[c] == '.':
                            tem1 = c
                            flg = 1
                    if flg == 0:
                        s = s[10:tem]
                    else:
                        s = s[tem1+1:tem]
                    s = s.replace(":",".")
                    uml.append(s)
                    start.append(s)
                    uml.append("->")
                    s1 = merge[k][j][5]
                    flg1 = 0
                    for c in range(len(s1)):
                        if s1[c] == '#':
                            tem = c
                        if s1[c] == '.':
                            tem1 = c
                            flg1 = 1
                    if flg1 == 0:
                        message = s1[tem+1:-1]
                        s1 = s1[8:tem]
                    else:
                        message = s1[tem+1:-1]
                        s1 = s1[tem1+1:tem]
                    s1 = s1.replace(":",".")
                    uml.append(s1)
                    uml.append(":")
                    uml.append(message)
                    uml.append("newline")
                    uml.append(f'activate {s1}')
                    uml.append("newline")
                if merge[k][j][3] == ' "Method Exit"':
                    s2 = merge[k][j][4]
                    flg2 = 0
                    for c in range(len(s2)):
                        if s2[c] == '#':
                            tem = c
                        if s2[c] == '.':
                            tem1 = c
                            flg2 = 1
                    if flg2 == 0:
                        s2 = s2[12:tem]
                    else:
                        s2 = s2[tem1+1:tem]
                    s2 = s2.replace(":",".")
                    end.append(s2)
                    uml.append(end[len(end) - 1])
                    uml.append("-->")
                    uml.append(start[len(start) - 1])
                    start.remove(start[len(start) - 1])
                    uml.append("newline")
                    uml.append(f'deactivate {end[len(end) - 1]}')
                    uml.append("newline")
            k += 1
        if k == len(merge) - 1:
            break
        #client exit server call 
        socket = []
        for o in range(2):
            if (temp+2) % client_length[client_counter] != 0:
                socket.append(merge[k][0][0])
                if o == 0:
                    socket.append("->")
                else:
                    socket.append(":sends request")
                    socket.append("newline")
            k += 1

        print('start1')
        print(start)
        print()
        print('end1')
        print(end)
        print()
        print('uml1')
        print(uml)

        temp += 1
        print(f'{temp}+')

['@startuml', 'SYSTEM', '->', 'ServerA', ':', 'main.1', 'newline', 'activate ServerA', 'newline']
start1
['SYSTEM', 'SYSTEM']

end1
[]

uml1
['@startuml', 'SYSTEM', '->', 'ServerA', ':', 'main.1', 'newline', 'activate ServerA', 'newline', 'SYSTEM', '->', 'ClientA2', ':', 'main:1', 'newline', 'activate ClientA2', 'newline']
1+
 " caller=ServerA#main:1
 " caller=SYSTEM
Merge list ServerA
Merge list ClientA2
start1
['SYSTEM', 'SYSTEM', 'SYSTEM']

end1
['ClientHandler.1']

uml1
['@startuml', 'SYSTEM', '->', 'ServerA', ':', 'main.1', 'newline', 'activate ServerA', 'newline', 'SYSTEM', '->', 'ClientA2', ':', 'main:1', 'newline', 'activate ClientA2', 'newline', 'ClientA2', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '->', 'ClientHandler.1', ':', 'ClientHandler.1', 'newline', 'activate ClientHandler.1', 'newline', 'ClientHandler.1', '-->', 'ServerA', 'newline', 'deactivate ClientHandler.1', 'newline', 'SYSTEM', '->', 'ClientHandler.1', ':', 'run.1', 'newline', 'activate ClientHand

In [ ]:
for m in range(2):
    socket.append(merge[k][0][0])
    if m == 0:
        socket.append("-->")
    else:
        socket.append(":sends response")
        socket.append("newline")

In [329]:
print(uml)
uml1 = []
def swapPositions(list, pos1, pos2): 
      
    list[pos1], list[pos2] = list[pos2], list[pos1] 
    return list

['@startuml', 'SYSTEM', '->', 'ServerA', ':', 'main:1', 'newline', 'activate ServerA', 'newline', 'SYSTE', '->', 'ClientA', ':', 'main:1', 'newline', 'activate ClientA', 'newline', 'ClientA', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '->', 'ClientHandler.1', ':', 'ClientHandler:1', 'newline', 'activate ClientHandler.1', 'newline', 'ClientHandler.1', '-->', 'ServerA', 'newline', 'deactivate ClientHandler.1', 'newline', 'SYSTEM', '->', 'ClientHandler.1', ':', 'run:1', 'newline', 'activate ClientHandler.1', 'newline', 'ClientA', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA', ':sends response', 'newline', 'ClientA', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA', ':sends response', 'newline', 'ClientA', '->', 'ServerA', ':sends request', 'newline', 'ClientA', '->', 'ServerA', ':sends request', 'newline', 'ServerA', '-->', 'ClientA', ':sends response', 'newline', 'ClientA', '->', 'ServerA', ':sends request', 'newline', 'S

In [330]:
# uml = ["@startuml","Alice","->","Bob",":","Hey","Bob","-->","Alice",":","Hi","@enduml"]
uml.append("@enduml")
uml1.append("@enduml")
print(start)
print()
print(end)

['SYSTE']

['ClientHandler.1', 'ClientHandler.2', 'ClientHandler.3', 'ClientA2', 'ClientHandler.3', 'ClientA1', 'ClientHandler.2', 'ClientA', 'ClientHandler.1']


In [331]:
with open('file.txt', 'w') as file:
    for u in uml:
        if u != "newline":
            file.write(u)
        if u == "@startuml" or u == "@enduml" or u == "newline":
            file.write("\n")

In [425]:
with open('file1.txt', 'w') as file:
    for u in uml1:
        if u != "newline":
            file.write(u)
        if u == "@startuml" or u == "newline":
            file.write("\n")